# Reusing old CVs

First, a refresher on disk-caching. Disk-caching is the term we use to indicate that the results of a CV have been stored. In practice, this makes analysis much faster, because not only do you not need to re-run the calculation, but you don't even need to load all the snapshot data (positions, etc) because all you need is the snapshot's unique identifier.

Because of the way disk-caching is integrated with storage, CVs are treated a little special in both netcdfplus and in SimStore. You need to do a few special things to use netcdfplus CVs with SimStore. Additionally, results from a netcdfplus CV will not be disk-cached in SimStore. However, it *is* still possible to use netcdfplus CVs with SimStore. This notebook will show how.

In [1]:
import openpathsampling as paths
from openpathsampling.experimental.storage import monkey_patch_saving
paths = monkey_patch_saving(paths)

## Reload from old storage

In [2]:
old_storage = paths.Storage("../toy_mstis_1k_OPS1_py36.nc", mode='r')

In [3]:
samples = old_storage.samplesets[-1]
scheme = old_storage.schemes[0]

## Create a SimStore storage, and run a simulation

In [4]:
from openpathsampling.experimental.simstore import SQLStorageBackend
from openpathsampling.experimental.storage import Storage

In [5]:
backend = SQLStorageBackend("storage_02.db", mode='w')
storage = Storage.from_backend(backend)

In [6]:
simulation = paths.PathSampling(storage=storage,
                                move_scheme=scheme,
                                sample_set=samples)
simulation.save_frequency = 50

In [7]:
# you'll notice a long delay on the first save point (50): that is 
# loading/resaving netcdfplus data
simulation.run(200)

Working on Monte Carlo cycle number 200
Running for 28 seconds -  0.15 seconds per step
Estimated time remaining: 0 seconds
DONE! Completed 200 Monte Carlo cycles.


In [8]:
old_storage.close()
storage.close()

## Reloading the storage

In [9]:
from openpathsampling.experimental.storage import monkey_patch_loading
paths = monkey_patch_loading(paths)

# note: If you don't need to load from a netcdfplus file, you can use 
# monkey_patch_all, which handles both saving and loading

In [10]:
backend = SQLStorageBackend("storage_02.db", mode='r')
storage = Storage.from_backend(backend)

In [11]:
%%time
sum([step.change.accepted for step in storage.steps])
# unfortunately, this will always be slow on first load with netcdfplus CVs,
# because they aren't compatible with lazy-loading trajectories

CPU times: user 1.69 s, sys: 40.6 ms, total: 1.73 s
Wall time: 1.75 s


130

In [12]:
cv_A = storage.cvs['opA']

In [13]:
cv_A(storage.steps[0].active[0])  # this fails without the monkey-patch

[0.19801861644370738,
 0.2018505255755742,
 0.20193459223958993,
 0.19891772975500482]